In [15]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local') \
    .appName('thailand') \
    .getOrCreate()

In [29]:
df = spark.read.parquet("archive/thailand_domestic_tourism_2019_2023_ver2.parquet")

In [30]:
df.show()

+-------------------+---------------+--------------------+-----------+----------+------------------+-----+
|               date|  province_thai|        province_eng|region_thai|region_eng|          variable|value|
+-------------------+---------------+--------------------+-----------+----------+------------------+-----+
|2019-01-01 00:00:00|  กรุงเทพมหานคร|             Bangkok|    ภาคกลาง|   central|ratio_tourist_stay|93.37|
|2019-01-01 00:00:00|         ลพบุรี|            Lopburi |    ภาคกลาง|   central|ratio_tourist_stay|61.32|
|2019-01-01 00:00:00|พระนครศรีอยุธยา|Phra Nakhon Si Ay...|    ภาคกลาง|   central|ratio_tourist_stay|73.37|
|2019-01-01 00:00:00|        สระบุรี|           Saraburi |    ภาคกลาง|   central|ratio_tourist_stay|67.33|
|2019-01-01 00:00:00|         ชัยนาท|            Chainat |    ภาคกลาง|   central|ratio_tourist_stay|79.31|
|2019-01-01 00:00:00|         นครปฐม|      Nakhon Pathom |    ภาคกลาง|   central|ratio_tourist_stay| 71.7|
|2019-01-01 00:00:00|      สิงห์บุรี|

In [31]:
df.dtypes

[('date', 'timestamp_ntz'),
 ('province_thai', 'string'),
 ('province_eng', 'string'),
 ('region_thai', 'string'),
 ('region_eng', 'string'),
 ('variable', 'string'),
 ('value', 'double')]

In [43]:
df_no_tourist_all = (df
                     .filter("variable='no_tourist_all'")
                     .select(
                         "date",
                         "province_eng",
                         "value",
                     )
                     .withColumnRenamed("value", "no_tourist_all")
                    )

df_revenue_all = (df
                  .filter("variable='revenue_all'")
                  .select(
                      "date",
                      "province_eng",
                      "value",
                  )
                  .withColumnRenamed("value", "revenue_all")           
                )
df_revenue_foreign = (df
                     .filter("variable='revenue_foreign'")
                     .select(
                         "date",
                         "province_eng",
                         "value",
                     )
                      .withColumnRenamed("value", "revenue_foreign")
                     )
df_no_tourist_foreign = (df
                         .filter("variable='no_tourist_foreign'")
                         .select(
                            "date",
                            "province_eng",
                            "value",
                         )
                         .withColumnRenamed("value","no_tourist_foreign")
    
)

In [44]:
df_revenue_foreign.show()

+-------------------+--------------------+---------------+
|               date|        province_eng|revenue_foreign|
+-------------------+--------------------+---------------+
|2019-01-01 00:00:00|             Bangkok|    5.218391E10|
|2019-01-01 00:00:00|            Lopburi |      5410000.0|
|2019-01-01 00:00:00|Phra Nakhon Si Ay...|       3.8448E8|
|2019-01-01 00:00:00|           Saraburi |         1.16E7|
|2019-01-01 00:00:00|            Chainat |       890000.0|
|2019-01-01 00:00:00|      Nakhon Pathom |        1.686E7|
|2019-01-01 00:00:00|          Sing Buri |       390000.0|
|2019-01-01 00:00:00|          Ang Thong |      5750000.0|
|2019-01-01 00:00:00|         Nonthaburi |       1.0509E8|
|2019-01-01 00:00:00|       Pathum Thani |       1.0378E8|
|2019-01-01 00:00:00|       Samut Prakan |       4.8786E8|
|2019-01-01 00:00:00|       Samut Sakhon |       730000.0|
|2019-01-01 00:00:00|       Chachoengsao |      4780000.0|
|2019-01-01 00:00:00|         Ratchaburi |        1.083E

In [45]:
df_final = (df_no_tourist_all
            .join(df_revenue_all, on=["date", "province_eng"])
            .join(df_revenue_foreign, on=["date", "province_eng"])
            .join(df_no_tourist_foreign, on=["date", "province_eng"])
            .selectExpr(
                "date",
                "province_eng",
                "no_tourist_foreign / no_tourist_all as no_percentage_of_foreign_tourists",
                "revenue_foreign / revenue_all as revenue_percentage_of_foreign_tourists",
            )
           )

In [47]:
df_final.show()

+-------------------+--------------------+---------------------------------+--------------------------------------+
|               date|        province_eng|no_percentage_of_foreign_tourists|revenue_percentage_of_foreign_tourists|
+-------------------+--------------------+---------------------------------+--------------------------------------+
|2019-01-01 00:00:00|             Bangkok|              0.40694470198814414|                    0.6369601578195282|
|2019-01-01 00:00:00|            Lopburi |             0.008795372658785695|                   0.01183186072959496|
|2019-01-01 00:00:00|Phra Nakhon Si Ay...|              0.23109584858330973|                     0.267235687029533|
|2019-01-01 00:00:00|           Saraburi |              0.02816113030554537|                   0.03335346042151873|
|2019-01-01 00:00:00|            Chainat |             0.007069416867957457|                  0.008743491502112191|
|2019-01-01 00:00:00|      Nakhon Pathom |             0.039509366546552

In [49]:
df_final.write.csv("chal4")